<a href="https://colab.research.google.com/github/nutworker/qM-AI-L/blob/main/QnA_Llama_3_1_8b_with_Unsloth_D2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
repository_url = 'https://github.com/nutworker/qM-AI-L'
!git clone {repository_url}

Cloning into 'qM-AI-L'...
remote: Enumerating objects: 558, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 558 (delta 96), reused 58 (delta 58), pack-reused 443 (from 1)
Receiving objects: 100% (558/558), 11.55 MiB | 2.46 MiB/s, done.
Resolving deltas: 100% (297/297), done.


In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [3]:
# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

# **Setting Up the Environment**

In [4]:
!pip install rouge_score
from rouge_score import rouge_scorer

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=9116cc7ec7bac8fa85c101cc305a1fa44610c653c74fb0ee96fc9d41a6f02449
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
# Importing Libraries

import os
import re
import pandas as pd # Data Handling
import torch
import tensorflow as tf
import numpy as np
import time
from google.colab import files
import shutil


In [6]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
from torch import __version__
from packaging.version import Version as V
if V(__version__) < V("2.4.0"):
  xformers = "xformers==0.0.27"
else:
  xformers =  "xformers"

In [8]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel, is_bfloat16_supported
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = 1024,
    dtype = None,
    load_in_4bit = True
)

==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

# **Downloading 4-bit Quantized Model and Adding LoRA Adapters**

# **Dataset-1 :Preparing the Dataset for Fine-tuning**

In [12]:
D2_train_path = '/content/qM-AI-L/QA-System/Dataset-2/train.csv'
D2_test_path = '/content/qM-AI-L/QA-System/Dataset-2/test.csv'
D2_dev_path = '/content/qM-AI-L/QA-System/Dataset-2/dev.csv'


In [13]:
D2_train_df = pd.read_csv(D2_train_path)
D2_test_df = pd.read_csv(D2_test_path)
D2_dev_df = pd.read_csv(D2_dev_path)


In [14]:
D2_train_df.shape, D2_test_df.shape, D2_dev_df.shape

((1985, 2), (249, 2), (248, 2))

In [15]:
D2_train_df.columns,  D2_test_df.columns,  D2_dev_df.columns

(Index(['question', 'answer'], dtype='object'),
 Index(['question', 'answer'], dtype='object'),
 Index(['question', 'answer'], dtype='object'))

## **Add prompt format column to train df for model input**

In [17]:
prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    prompt.format(
        "answer for this question", # instruction
        "Is 9.11 larger than 9.9?", # input
        "", # output—leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
answer for this question

### Input:
Is 9.11 larger than 9.9?

### Response:
No

### Explanation:
9.11 is smaller than 9.9
<|end_of_text|>


In [18]:
EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN

'<|end_of_text|>'

In [19]:
BOS_TOKEN = tokenizer.bos_token
BOS_TOKEN

'<|begin_of_text|>'

In [20]:
#Function for Train dataset
EOS_TOKEN = tokenizer.eos_token
def formatted_train(x):

  instruction = "Generate an answer for the following question in less than three sentences."

  if x['question']:
    formatted_text = f""" {BOS_TOKEN} Below is an instruction that describes a task. \
    Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    {instruction}

    ### Input:
    {x['question']}

    ### Response:
    {x['answer']} {EOS_TOKEN}"""

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Response:
    {x['answer']} {EOS_TOKEN}"""

  return formatted_text

In [ ]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatted_train(x):
#     instructions = "Generate an answer for the following question in less than three sentences."
#     inputs       = x['question']
#     outputs      = x['answer1']
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }

In [21]:
#Function for Test and Dev dataset
def format_test(x):
  instruction = "Generate an answer for the following question in less than three sentences."

  if x['question']:
    formatted_text = f""" {BOS_TOKEN} Below is an instruction that describes a task. \
    Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    {instruction}

    ### Input:
    {x['question']}

    ### Response:
     {EOS_TOKEN}"""

  else:
    formatted_text = f"""Below is an instruction that describes a task. \
    Write a response that appropriately completes the request.

    ### Instruction:
    {instruction}

    ### Response:
     {EOS_TOKEN}"""

  return formatted_text

In [22]:
D2_train_df

,question,answer
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor..."
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...
...,...,...
1980,What is the method of vectorization in Bag-of-...,The simplest method is to mark the presence of...
1981,What is the significance of feature interactions?,Feature interactions capture relationships bet...
1982,What is the difference between hard margin and...,Hard margin SVM assumes that the data is linea...
1983,How does quantization error change with differ...,Quantization error decreases with larger data ...


In [23]:
D2_train_df["formatted_text"] = D2_train_df.apply(formatted_train, axis=1)
D2_test_df["formatted_text"] = D2_test_df.apply(format_test, axis=1)
D2_dev_df["formatted_text"] = D2_dev_df.apply(format_test, axis=1)

**Check train prompt includes answer column/ data, and test and dev prompts does not include answers/targets**

In [24]:
D2_train_df

,question,answer,formatted_text
0,What is the role of Siamese networks in domain...,Siamese networks can be used to align the repr...,<|begin_of_text|> Below is an instruction tha...
1,What is a recurrent neural network (RNN)?,"An RNN handles sequence data, capturing tempor...",<|begin_of_text|> Below is an instruction tha...
2,What are Eigen values and Eigen vectors in the...,Eigen values represent the variance captured b...,<|begin_of_text|> Below is an instruction tha...
3,What are Deep Q-Networks?,Deep Q-networks are neural networks that appro...,<|begin_of_text|> Below is an instruction tha...
4,What properties of words can Word2Vec extract?,Word2Vec can extract and provide the most simi...,<|begin_of_text|> Below is an instruction tha...
...,...,...,...
1980,What is the method of vectorization in Bag-of-...,The simplest method is to mark the presence of...,<|begin_of_text|> Below is an instruction tha...
1981,What is the significance of feature interactions?,Feature interactions capture relationships bet...,<|begin_of_text|> Below is an instruction tha...
1982,What is the difference between hard margin and...,Hard margin SVM assumes that the data is linea...,<|begin_of_text|> Below is an instruction tha...
1983,How does quantization error change with differ...,Quantization error decreases with larger data ...,<|begin_of_text|> Below is an instruction tha...


In [25]:
D2_train_df['formatted_text'][1]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is a recurrent neural network (RNN)?\n\n    ### Response:\n    An RNN handles sequence data, capturing temporal dependencies with cycles in connections. <|end_of_text|>'

In [26]:
D2_test_df['formatted_text'][0]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is feature selection?\n\n    ### Response:\n     <|end_of_text|>'

In [27]:
D1_dev_df['formatted_text'][0]

NameError: name 'D1_dev_df' is not defined

# **Convert DFs to Datasets**

In [28]:
!pip install datasets

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [31]:
from datasets import Dataset
D2_Train_dataset = Dataset.from_pandas(D2_train_df)
D2_Test_dataset = Dataset.from_pandas(D2_test_df)
D2_Dev_dataset = Dataset.from_pandas(D2_dev_df)

In [32]:
D2_Train_dataset, D2_Test_dataset, D2_Dev_dataset

(Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 1985
 }),
 Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 249
 }),
 Dataset({
     features: ['question', 'answer', 'formatted_text'],
     num_rows: 248
 }))

In [33]:
#check train dataset
D2_Train_dataset['formatted_text'][0:2]

[' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is the role of Siamese networks in domain adaptation?\n\n    ### Response:\n    Siamese networks can be used to align the representations of source and target domains by minimizing the discrepancy between them, making the model more robust to domain shifts. <|end_of_text|>',
 ' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is a recurrent neural network (RNN)?\n\n    ### Response:\

# **Fine-tuning Google Gemma for Code Dataset**


In [34]:
from trl import SFTTrainer
from transformers import TrainingArguments

In [61]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = True, # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [64]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = D2_Train_dataset,
    dataset_text_field = "formatted_text",
    max_seq_length = 1024,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 10,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "paged_adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/1985 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [65]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,985 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 10
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,0.697600
2,0.771000
3,0.785300
4,0.811700
5,0.732500
6,0.621700
7,0.732700
8,0.636300
9,0.665500
10,0.795200


**#check test data for infrencing and answer generation**

In [66]:
from transformers import TextStreamer

In [67]:
D2_test_df['formatted_text'][2], D2_test_df['formatted_text'][3]

(' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is batch normalization and how does it work?\n\n    ### Response:\n     <|end_of_text|>',
 ' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    How does Word2Vec create word embeddings?\n\n    ### Response:\n     <|end_of_text|>')

# **Checkout the trained model generating answer**

In [68]:
Prompt = D2_test_df['formatted_text'][45]


FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    Prompt
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
# Get the generated text by decoding the IDs
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text


<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

    ### Input:
    What are the components of a confusion matrix?

    ### Response:
     <|end_of_text|><|begin_of_text|> <|begin_of_text|>:// A confusion matrix provides a summary of the performance of a classification model. It contains four elements: true positive, false positive, true negative, and false negative. <|end_of_text|>


# **Function to extract answer part from the model generated response**

In [69]:
generated_text

'  Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What are the components of a confusion matrix?\n\n    ### Response:\n      :// A confusion matrix provides a summary of the performance of a classification model. It contains four elements: true positive, false positive, true negative, and false negative. '

In [70]:

# **Function to extract answer part from the model generated response**
def get_answer(text):
    start_tag = "### Response:"

    # Find the start and end indices
    start_idx = text.find(start_tag)

    # Check if both tags are found
    if start_idx == -1:
        return None  # Tags not found

    # Extract content between the tags
    answer = text[start_idx + len(start_tag):].strip()

    return answer

get_answer(generated_text)

':// A confusion matrix provides a summary of the performance of a classification model. It contains four elements: true positive, false positive, true negative, and false negative.'

In [71]:
D2_test_df.columns, D2_test_df.shape

(Index(['question', 'answer', 'formatted_text'], dtype='object'), (249, 3))

In [76]:
# prompt: slice D2_test_df to 100 rows randomly

D2_test_df_sample = D2_test_df.sample(n=20, random_state=42)


In [83]:
D2_test_df_sample['formatted_text'][6]

' <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.\n\n    ### Instruction:\n    Generate an answer for the following question in less than three sentences.\n\n    ### Input:\n    What is variance in machine learning?\n\n    ### Response:\n     <|end_of_text|>'

In [85]:
original_answers = []
llama_answers = []

for index, row in D2_test_df_sample.iterrows():
    original_answer = row['answer']


    Prompt = row['formatted_text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
    # Get the generated text by decoding the IDs
    llama_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    llama_answers.append(get_answer(llama_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, llama_answers))

D2_Test_Llama_with_Unsloth_df = pd.DataFrame(zipped_subjects, columns = ['original_answers', 'Llama_answers'])
D2_Test_Llama_with_Unsloth_df

<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

    ### Input:
    How do word embeddings capture semantic relationships between words?

    ### Response:
     <|end_of_text|><|begin_of_text|>1n2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 16

,original_answers,Llama_answers
0,Words with similar meanings are represented by...,1n2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,Variance refers to the error introduced by the...,Variance is a measure of the spread of the dat...
2,A cross-validation score is the performance me...,:// A cross-validation score is a metric used ...
3,PEFT refers to techniques that allow fine-tuni...,"php require_once(""php/answer.php"");?> Paramete..."
4,Feature selection helps prevent overfitting by...,php echo htmlspecialchars($response);? Featur...
5,A fully connected layer connects each neuron t...,"php echo "" A fully connected layer in deep le..."
6,"GPT-3 is an advanced version of GPT-2, with 17...",":// GPT-3 is a more powerful version of GPT-2,..."
7,TF-IDF is the product of TF and IDF.,:// TF-IDF is calculated by multiplying the t...
8,Feature engineering for unstructured data invo...,:// Feature engineering for unstructured da...
9,Eigenvalues and eigenvectors are scalar values...,2nbsp;Eigenvalues are the values that are asso...


In [89]:
# prompt: Using dataframe D1_Test_Gemma_with_Unsloth_df: get all rows which that are not None in Llama_answers

# Select rows where Llama_answers is not null
D2_Test_Llama3_with_Unsloth = D2_Test_Llama_with_Unsloth_df[D2_Test_Llama_with_Unsloth_df['Llama_answers'].notna()].reset_index()
D2_Test_Llama3_with_Unsloth


,index,original_answers,Llama_answers
0,0,Words with similar meanings are represented by...,1n2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,1,Variance refers to the error introduced by the...,Variance is a measure of the spread of the dat...
2,2,A cross-validation score is the performance me...,:// A cross-validation score is a metric used ...
3,3,PEFT refers to techniques that allow fine-tuni...,"php require_once(""php/answer.php"");?> Paramete..."
4,4,Feature selection helps prevent overfitting by...,php echo htmlspecialchars($response);? Featur...
5,5,A fully connected layer connects each neuron t...,"php echo "" A fully connected layer in deep le..."
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",":// GPT-3 is a more powerful version of GPT-2,..."
7,7,TF-IDF is the product of TF and IDF.,:// TF-IDF is calculated by multiplying the t...
8,8,Feature engineering for unstructured data invo...,:// Feature engineering for unstructured da...
9,9,Eigenvalues and eigenvectors are scalar values...,2nbsp;Eigenvalues are the values that are asso...


# ** Evaluate model performance- Rouge**

In [86]:
!pip install rouge-score  # Install the rouge-score library
from rouge_score import rouge_scorer

In [90]:
D2_test_scores = D2_Test_Llama3_with_Unsloth.copy()

In [92]:
from rouge_score import rouge_scorer

def calculate_rouge(reference, hypothesis):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True) # Instantiate a RougeScorer object
    scores = scorer.score(reference, hypothesis) # Call the score method on the RougeScorer object
    return {metric: score.fmeasure for metric, score in scores.items()}

# Apply ROUGE score calculation to each row
D2_test_scores['rouge_scores'] = D2_test_scores.apply(lambda row: calculate_rouge(row['original_answers'], row['Llama_answers']), axis=1)

In [93]:
D2_test_scores

,index,original_answers,Llama_answers,rouge_scores
0,0,Words with similar meanings are represented by...,1n2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ..."
1,1,Variance refers to the error introduced by the...,Variance is a measure of the spread of the dat...,"{'rouge1': 0.32558139534883723, 'rouge2': 0.0,..."
2,2,A cross-validation score is the performance me...,:// A cross-validation score is a metric used ...,"{'rouge1': 0.4383561643835616, 'rouge2': 0.225..."
3,3,PEFT refers to techniques that allow fine-tuni...,"php require_once(""php/answer.php"");?> Paramete...","{'rouge1': 0.29090909090909095, 'rouge2': 0.03..."
4,4,Feature selection helps prevent overfitting by...,php echo htmlspecialchars($response);? Featur...,"{'rouge1': 0.5777777777777778, 'rouge2': 0.325..."
5,5,A fully connected layer connects each neuron t...,"php echo "" A fully connected layer in deep le...","{'rouge1': 0.37037037037037035, 'rouge2': 0.19..."
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",":// GPT-3 is a more powerful version of GPT-2,...","{'rouge1': 0.4745762711864407, 'rouge2': 0.315..."
7,7,TF-IDF is the product of TF and IDF.,:// TF-IDF is calculated by multiplying the t...,"{'rouge1': 0.20512820512820512, 'rouge2': 0.10..."
8,8,Feature engineering for unstructured data invo...,:// Feature engineering for unstructured da...,"{'rouge1': 0.5909090909090909, 'rouge2': 0.428..."
9,9,Eigenvalues and eigenvectors are scalar values...,2nbsp;Eigenvalues are the values that are asso...,"{'rouge1': 0.5263157894736842, 'rouge2': 0.111..."


In [94]:
pd.json_normalize(D2_test_scores['rouge_scores'])

,rouge1,rouge2,rougeL,rougeLsum
0,0.000000,0.000000,0.000000,0.000000
1,0.325581,0.000000,0.232558,0.232558
2,0.438356,0.225352,0.328767,0.328767
3,0.290909,0.037736,0.218182,0.218182
4,0.577778,0.325581,0.444444,0.444444
5,0.370370,0.192308,0.370370,0.370370
6,0.474576,0.315789,0.440678,0.440678
7,0.205128,0.108108,0.205128,0.205128
8,0.590909,0.428571,0.590909,0.590909
9,0.526316,0.111111,0.368421,0.368421


In [95]:
D2_test_scores = pd.concat([D2_test_scores, pd.json_normalize(D2_test_scores['rouge_scores'])], axis=1)


In [96]:
D2_test_scores

,index,original_answers,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum
0,0,Words with similar meanings are represented by...,1n2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000
1,1,Variance refers to the error introduced by the...,Variance is a measure of the spread of the dat...,"{'rouge1': 0.32558139534883723, 'rouge2': 0.0,...",0.325581,0.000000,0.232558,0.232558
2,2,A cross-validation score is the performance me...,:// A cross-validation score is a metric used ...,"{'rouge1': 0.4383561643835616, 'rouge2': 0.225...",0.438356,0.225352,0.328767,0.328767
3,3,PEFT refers to techniques that allow fine-tuni...,"php require_once(""php/answer.php"");?> Paramete...","{'rouge1': 0.29090909090909095, 'rouge2': 0.03...",0.290909,0.037736,0.218182,0.218182
4,4,Feature selection helps prevent overfitting by...,php echo htmlspecialchars($response);? Featur...,"{'rouge1': 0.5777777777777778, 'rouge2': 0.325...",0.577778,0.325581,0.444444,0.444444
5,5,A fully connected layer connects each neuron t...,"php echo "" A fully connected layer in deep le...","{'rouge1': 0.37037037037037035, 'rouge2': 0.19...",0.370370,0.192308,0.370370,0.370370
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",":// GPT-3 is a more powerful version of GPT-2,...","{'rouge1': 0.4745762711864407, 'rouge2': 0.315...",0.474576,0.315789,0.440678,0.440678
7,7,TF-IDF is the product of TF and IDF.,:// TF-IDF is calculated by multiplying the t...,"{'rouge1': 0.20512820512820512, 'rouge2': 0.10...",0.205128,0.108108,0.205128,0.205128
8,8,Feature engineering for unstructured data invo...,:// Feature engineering for unstructured da...,"{'rouge1': 0.5909090909090909, 'rouge2': 0.428...",0.590909,0.428571,0.590909,0.590909
9,9,Eigenvalues and eigenvectors are scalar values...,2nbsp;Eigenvalues are the values that are asso...,"{'rouge1': 0.5263157894736842, 'rouge2': 0.111...",0.526316,0.111111,0.368421,0.368421


In [97]:
!pip install nltk sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 9.5 MB/s eta 0:00:00


In [98]:
import sacrebleu
from nltk.translate import meteor_score
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [99]:
# Tokenization function
def tokenize(text):
    return text.lower().split()


In [100]:
def calculate_bleu(reference, hypothesis):
    # BLEU requires tokenized input for reference, but not for hypothesis
    reference = [tokenize(reference)]
    bleu = sacrebleu.corpus_bleu([hypothesis], reference) # hypothesis should be a string
    return bleu.score


D2_test_scores['bleu_score'] = D2_test_scores.apply(lambda row: calculate_bleu(row['original_answers'], row['Llama_answers']), axis=1)

# Calculate METEOR score
def calculate_meteor(reference, hypothesis):
    # Tokenize both reference and hypothesis
    tokenized_reference = tokenize(reference)
    tokenized_hypothesis = tokenize(hypothesis)
    return meteor_score.single_meteor_score(tokenized_reference, tokenized_hypothesis)  # Pass tokenized inputs

D2_test_scores['meteor_score'] = D2_test_scores.apply(lambda row: calculate_meteor(row['original_answers'], row['Llama_answers']), axis=1)

In [101]:
D2_test_scores

,index,original_answers,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum,bleu_score,meteor_score
0,0,Words with similar meanings are represented by...,1n2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...,"{'rouge1': 0.0, 'rouge2': 0.0, 'rougeL': 0.0, ...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,Variance refers to the error introduced by the...,Variance is a measure of the spread of the dat...,"{'rouge1': 0.32558139534883723, 'rouge2': 0.0,...",0.325581,0.000000,0.232558,0.232558,0.000000,0.149573
2,2,A cross-validation score is the performance me...,:// A cross-validation score is a metric used ...,"{'rouge1': 0.4383561643835616, 'rouge2': 0.225...",0.438356,0.225352,0.328767,0.328767,0.700277,0.353942
3,3,PEFT refers to techniques that allow fine-tuni...,"php require_once(""php/answer.php"");?> Paramete...","{'rouge1': 0.29090909090909095, 'rouge2': 0.03...",0.290909,0.037736,0.218182,0.218182,0.000000,0.076336
4,4,Feature selection helps prevent overfitting by...,php echo htmlspecialchars($response);? Featur...,"{'rouge1': 0.5777777777777778, 'rouge2': 0.325...",0.577778,0.325581,0.444444,0.444444,0.000000,0.407306
5,5,A fully connected layer connects each neuron t...,"php echo "" A fully connected layer in deep le...","{'rouge1': 0.37037037037037035, 'rouge2': 0.19...",0.370370,0.192308,0.370370,0.370370,0.813035,0.456349
6,6,"GPT-3 is an advanced version of GPT-2, with 17...",":// GPT-3 is a more powerful version of GPT-2,...","{'rouge1': 0.4745762711864407, 'rouge2': 0.315...",0.474576,0.315789,0.440678,0.440678,0.000000,0.340979
7,7,TF-IDF is the product of TF and IDF.,:// TF-IDF is calculated by multiplying the t...,"{'rouge1': 0.20512820512820512, 'rouge2': 0.10...",0.205128,0.108108,0.205128,0.205128,0.000000,0.147059
8,8,Feature engineering for unstructured data invo...,:// Feature engineering for unstructured da...,"{'rouge1': 0.5909090909090909, 'rouge2': 0.428...",0.590909,0.428571,0.590909,0.590909,0.000000,0.519693
9,9,Eigenvalues and eigenvectors are scalar values...,2nbsp;Eigenvalues are the values that are asso...,"{'rouge1': 0.5263157894736842, 'rouge2': 0.111...",0.526316,0.111111,0.368421,0.368421,1.646664,0.322625


In [102]:
average_scores = {
    'average_rouge1': D2_test_scores['rouge1'].mean(),
    'average_rouge2': D2_test_scores['rouge2'].mean(),
    'average_rougeL': D2_test_scores['rougeL'].mean(),
    'average_rougeLsum': D2_test_scores['rougeLsum'].mean(),
    'average_bleu_score': D2_test_scores['bleu_score'].mean(),
    'average_meteor_score': D2_test_scores['meteor_score'].mean(),
}

In [103]:
print("Average Scores for Dataset-2 (Test-Answer-1) QnA task using LLaMA 8B_with_Unsloth model:\n")
for metric, score in average_scores.items():
    print(f"{metric}: {score:.4f}")

Average Scores for Dataset-2 (Test-Answer-1) QnA task using LLaMA 8B_with_Unsloth model:

average_rouge1: 0.4036
average_rouge2: 0.2035
average_rougeL: 0.3459
average_rougeLsum: 0.3459
average_bleu_score: 0.4513
average_meteor_score: 0.2853


# **Repeat above evaluation process for the 2nd Answer annotation**

In [ ]:
original_answers = []
llama_answers = []

for index, row in D1_test_df.iterrows():
    original_answer = row['answer2']


    Prompt = row['formatted_text']

    # Define the device (GPU if available, otherwise CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move both models to the same device
    model.to(device)
    inputs = tokenizer(
    [
        Prompt
    ], return_tensors = "pt").to("cuda")

    text_streamer = TextStreamer(tokenizer)
    generated_ids = model.generate(**inputs, streamer=text_streamer, max_new_tokens=512)
    # Get the generated text by decoding the IDs
    llama_answer = tokenizer.decode(generated_ids[0], skip_special_tokens=True) # Apply to the decoded text
    llama_answers.append(get_answer(llama_answer))


    original_answers.append(original_answer)

zipped_subjects = list(zip(original_answers, llama_answers))

D1_Test_llama_with_Unsloth_df_A2 = pd.DataFrame(zipped_subjects, columns = ['answer-2', 'Llama_answers'])
D1_Test_llama_with_Unsloth_df_A2

<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

    ### Input:
    How we can effectively convert 2D images to 1D?

    ### Response:
     <|end_of_text|><|begin_of_text|>://We can use different methods to convert 2D images to 1D, such as flattening the image by reshaping it to a single row or column, extracting features using CNNs, or applying dimensionality reduction techniques like PCA. <|end_of_text|>
<|begin_of_text|> <|begin_of_text|> Below is an instruction that describes a task.     Write a response that appropriately completes the request. Provide a answer without including any reference links or citations.

    ### Instruction:
    Generate an answer for the following question in less than three sentences.

   

,answer-2,Llama_answers
0,"To effectively convert 2D images to 1D, use te...",://We can use different methods to convert 2D ...
1,"Yes, autoencoders can be used for dimensionali...",":// Yes, autoencoders can be used for dimensio..."
2,NLP models struggle with tasks that require re...,:// The current challenge in NLP is to make i...
3,"Neural networks are powerful, but they may str...",Neural networks are capable of solving a wide ...
4,Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM to avoid ove..."
...,...,...
115,Machine learning is a subset of data mining th...,Machine learning and data mining are related b...
116,CLAMP is a comprehensive clinical Natural Lang...,"No, there is no software specifically designed..."
117,Slicing can be useful for working with large d...,Slicing is typically used to extract a portion...
118,"Yes,Unlike stemming, which simply trims words ...",":// In general, lemmatization is considered to..."


In [ ]:
D1_test_scores_A2 = D1_Test_llama_with_Unsloth_df_A2.copy()

In [ ]:
D1_test_scores_A2['rouge_scores'] = D1_test_scores_A2.apply(lambda row: calculate_rouge(row['answer-2'], row['Llama_answers']), axis=1)

In [ ]:
D1_test_scores_A2 = pd.concat([D1_test_scores_A2, pd.json_normalize(D1_test_scores_A2['rouge_scores'])], axis=1)

In [ ]:
D1_test_scores_A2

,answer-2,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum
0,"To effectively convert 2D images to 1D, use te...",://We can use different methods to convert 2D ...,"{'rouge1': 0.42424242424242425, 'rouge2': 0.18...",0.424242,0.187500,0.272727,0.272727
1,"Yes, autoencoders can be used for dimensionali...",":// Yes, autoencoders can be used for dimensio...","{'rouge1': 0.5882352941176471, 'rouge2': 0.448...",0.588235,0.448980,0.549020,0.549020
2,NLP models struggle with tasks that require re...,:// The current challenge in NLP is to make i...,"{'rouge1': 0.0975609756097561, 'rouge2': 0.0, ...",0.097561,0.000000,0.097561,0.097561
3,"Neural networks are powerful, but they may str...",Neural networks are capable of solving a wide ...,"{'rouge1': 0.22222222222222224, 'rouge2': 0.09...",0.222222,0.093023,0.222222,0.222222
4,Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM to avoid ove...","{'rouge1': 0.20408163265306123, 'rouge2': 0.0,...",0.204082,0.000000,0.204082,0.204082
...,...,...,...,...,...,...,...
115,Machine learning is a subset of data mining th...,Machine learning and data mining are related b...,"{'rouge1': 0.393939393939394, 'rouge2': 0.0625...",0.393939,0.062500,0.212121,0.212121
116,CLAMP is a comprehensive clinical Natural Lang...,"No, there is no software specifically designed...","{'rouge1': 0.2352941176470588, 'rouge2': 0.0, ...",0.235294,0.000000,0.176471,0.176471
117,Slicing can be useful for working with large d...,Slicing is typically used to extract a portion...,"{'rouge1': 0.375, 'rouge2': 0.0, 'rougeL': 0.3...",0.375000,0.000000,0.312500,0.312500
118,"Yes,Unlike stemming, which simply trims words ...",":// In general, lemmatization is considered to...","{'rouge1': 0.18181818181818182, 'rouge2': 0.0,...",0.181818,0.000000,0.109091,0.109091


In [ ]:
D1_test_scores_A2['bleu_score'] = D1_test_scores_A2.apply(lambda row: calculate_bleu(row['answer-2'], row['Llama_answers']), axis=1)


D1_test_scores_A2['meteor_score'] = D1_test_scores_A2.apply(lambda row: calculate_meteor(row['answer-2'], row['Llama_answers']), axis=1)

In [ ]:
D1_test_scores_A2

,answer-2,Llama_answers,rouge_scores,rouge1,rouge2,rougeL,rougeLsum,bleu_score,meteor_score
0,"To effectively convert 2D images to 1D, use te...",://We can use different methods to convert 2D ...,"{'rouge1': 0.42424242424242425, 'rouge2': 0.18...",0.424242,0.187500,0.272727,0.272727,0.852245,0.374150
1,"Yes, autoencoders can be used for dimensionali...",":// Yes, autoencoders can be used for dimensio...","{'rouge1': 0.5882352941176471, 'rouge2': 0.448...",0.588235,0.448980,0.549020,0.549020,1.506189,0.371444
2,NLP models struggle with tasks that require re...,:// The current challenge in NLP is to make i...,"{'rouge1': 0.0975609756097561, 'rouge2': 0.0, ...",0.097561,0.000000,0.097561,0.097561,0.000000,0.054348
3,"Neural networks are powerful, but they may str...",Neural networks are capable of solving a wide ...,"{'rouge1': 0.22222222222222224, 'rouge2': 0.09...",0.222222,0.093023,0.222222,0.222222,0.000000,0.144788
4,Scaling the input data is advisable when utili...,"Yes, scaling is necessary for SVM to avoid ove...","{'rouge1': 0.20408163265306123, 'rouge2': 0.0,...",0.204082,0.000000,0.204082,0.204082,2.627962,0.074184
...,...,...,...,...,...,...,...,...,...
115,Machine learning is a subset of data mining th...,Machine learning and data mining are related b...,"{'rouge1': 0.393939393939394, 'rouge2': 0.0625...",0.393939,0.062500,0.212121,0.212121,0.918708,0.264684
116,CLAMP is a comprehensive clinical Natural Lang...,"No, there is no software specifically designed...","{'rouge1': 0.2352941176470588, 'rouge2': 0.0, ...",0.235294,0.000000,0.176471,0.176471,0.000000,0.091743
117,Slicing can be useful for working with large d...,Slicing is typically used to extract a portion...,"{'rouge1': 0.375, 'rouge2': 0.0, 'rougeL': 0.3...",0.375000,0.000000,0.312500,0.312500,0.000000,0.198864
118,"Yes,Unlike stemming, which simply trims words ...",":// In general, lemmatization is considered to...","{'rouge1': 0.18181818181818182, 'rouge2': 0.0,...",0.181818,0.000000,0.109091,0.109091,1.335353,0.096463


In [ ]:
A2_average_scores = {
    'average_rouge1': D1_test_scores_A2['rouge1'].mean(),
    'average_rouge2': D1_test_scores_A2['rouge2'].mean(),
    'average_rougeL': D1_test_scores_A2['rougeL'].mean(),
    'average_rougeLsum': D1_test_scores_A2['rougeLsum'].mean(),
    'average_bleu_score': D1_test_scores_A2['bleu_score'].mean(),
    'average_meteor_score': D1_test_scores_A2['meteor_score'].mean(),
}

In [ ]:
print("Average Scores for Dataset-1 (Test-Answer-2) QnA task using Gemma_7b_with_Unsloth model:\n")
for metric, score in A2_average_scores.items():
    print(f"{metric}: {score:.4f}")


Average Scores for Dataset-1 (Test-Answer-2) QnA task using Gemma_7b_with_Unsloth model:

average_rouge1: 0.3766
average_rouge2: 0.1439
average_rougeL: 0.2885
average_rougeLsum: 0.2918
average_bleu_score: 0.6694
average_meteor_score: 0.2329


In [ ]:
# make a df with A2_average_scores and average_scores

# Create a DataFrame from the dictionaries
D1_Test_Scores = pd.DataFrame([average_scores, A2_average_scores], index=['Llama3-8b_model Vs.Answer-1', 'Llama3-8b_model Vs.Answer-2'])

# Transpose the DataFrame for better readability
D1_Test_Scores = D1_Test_Scores.T

D1_Test_Scores


,Llama3-8b_model Vs.Answer-1,Llama3-8b_model Vs.Answer-2
average_rouge1,0.400487,0.376601
average_rouge2,0.177318,0.143933
average_rougeL,0.325214,0.288521
average_rougeLsum,0.328088,0.291801
average_bleu_score,0.693772,0.669431
average_meteor_score,0.268180,0.232883
